In [4]:
import os
import requests
from datetime import datetime
import pandas as pd
from googlesearch import search
from bs4 import BeautifulSoup
from urllib.parse import urljoin

In [5]:
class ImageCoinCrawler:
    def __init__(self):
        self.download_folder = 'coin_images'

        if not os.path.exists('visited_pages.csv'):
            self.visited_pages = []
        else:
            self.visited_pages = pd.read_csv('visited_pages.csv')['0'].values.tolist()

        if not os.path.exists('image_urls.csv'):
            self.image_urls = []
        else:
            self.image_urls = pd.read_csv('image_urls.csv')['0'].values.tolist()

    def is_valid_image_url(self, url):
        image_extensions = ['.jpg', '.jpeg', '.png', '.gif']
        return any(url.lower().endswith(ext) for ext in image_extensions)
    
    def is_valid_image_name(self, url):
        image_contains = ['moeda', 'moedas', 'coin', 'centavo', 'centavos', 'real', 'reais', 'dinheiro', 'money', 'currency']
        return any(nm in url.lower() for nm in image_contains)
    
    def update_visited_pages(self):
        pd.DataFrame(self.visited_pages).to_csv('visited_pages.csv', index=False)

    def update_image_urls(self):
        pd.DataFrame(self.image_urls).to_csv('image_urls.csv', index=False)

    def get_image_urls(self, query, num_results=50):

        results = search(query, num_results=num_results, sleep_interval=15, timeout=3, lang='pt')
        for page_url in results:
            try:
                if page_url in self.visited_pages:
                    continue
                
                print('loading page: ', page_url, '...')
                self.visited_pages.append(page_url)

                response = requests.get(page_url)
                soup = BeautifulSoup(response.text, 'html.parser')

                for img_tag in soup.find_all('img'):
                    img_url = img_tag.get('src')

                    if not self.is_valid_image_url(img_url):
                        continue
                    
                    full_img_url = urljoin(page_url, img_url)
                    if full_img_url not in self.image_urls:
                        print('---------- found image: ', full_img_url)
                        self.image_urls.append(full_img_url)
                        self.update_image_urls()

                self.update_visited_pages()
            except:
                continue

    def download_images(self, urls):
        if not os.path.exists(self.download_folder):
            os.makedirs(self.download_folder)

        for i, url in enumerate(urls):
            try:
                response = requests.get(url)
                file_ext = os.path.splitext(url)[-1].split('?')[0]
                file_path = os.path.join(self.download_folder, f'coin_{i}{file_ext}')
                with open(file_path, 'wb') as f:
                    f.write(response.content)
            except:
                continue

In [6]:
coin_query = 'moedas brasileiras'
num_images = 150

imageCoin = ImageCoinCrawler()
imageCoin.get_image_urls(coin_query, num_results=num_images)

imageCoin.download_images(imageCoin.image_urls)

HTTPError: 429 Client Error: Too Many Requests for url: https://www.google.com/sorry/index?continue=https://www.google.com/search%3Fq%3Dmoedas%252Bbrasileiras%26num%3D152%26hl%3Dpt%26start%3D0&hl=pt&q=EgS9BvjtGOfo4KEGIjBhIfaVlNr0DyhCESOGeoKJnpNUBYqGdte3pfOrsNTfgJTZHVOvhS14ifVMspb1MhgyAXI